In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from datetime import datetime, timedelta

# input file paths
input_files = [
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock A1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock B1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock C1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock D1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock E1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock F1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock G1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock H1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock I1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock J1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock K1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock L1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock M1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock N1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock O1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock P1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock Q1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock R1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock S1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock T1.xlsx"
]


for file_path in input_files:

    file_name = file_path.split("/")[-1].split(".")[0]

    print(f"Processing file: {file_name}")

    # Load the data
    data = pd.read_excel(file_path, sheet_name="Sheet1")

    # Preprocess the Data
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # features to scale
    features_to_scale = ['Close', 'Open', 'High', 'Low', 'Volume']

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[features_to_scale])

    # Training
    def create_sequences(data, window_size=60):
        X, y = [], []
        for i in range(len(data) - window_size):
            X.append(data[i:i+window_size])
            y.append(data[i+window_size, 0])
        return np.array(X), np.array(y)

    window_size = 60
    X, y = create_sequences(scaled_data)

    # Split Data
    train_size = int(len(X) * 0.8)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]

    # LSTM Model
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        LSTM(32),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Train
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )

    # Generate Predictions

    # trading dates for 2023
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    trading_dates_2023 = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)
                           if (start_date + timedelta(days=i)).weekday() < 5]  # Exclude weekends

    predictions = []
    last_window = X[-1]

    for _ in range(len(trading_dates_2023)):
        pred = model.predict(np.array([last_window]))[0][0]
        predictions.append(pred)


        last_window = np.roll(last_window, -1, axis=0)
        last_window[-1][0] = pred


    dummy_array = np.zeros((len(predictions), len(features_to_scale)))
    dummy_array[:, 0] = np.array(predictions)

    predicted_prices = scaler.inverse_transform(dummy_array)[:, 0]

    # Save Predictions
    output_file_name = f"{file_name}_predicted_prices_2023.xlsx"

    output_df = pd.DataFrame({
        "Trading Date": trading_dates_2023,
        "Predicted Close": predicted_prices
    })

    output_df.to_excel(output_file_name, index=False)

    print(f"Predictions saved to {output_file_name}")


Processing file: Stock A1
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - loss: 0.0479 - val_loss: 0.0052
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0038 - val_loss: 0.0051
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0032 - val_loss: 0.0047
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0027 - val_loss: 0.0044
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0025 - val_loss: 0.0049
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - loss: 0.0025 - val_loss: 0.0055
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - loss: 0.0024 - val_loss: 0.0054
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.0022 - val_loss: 0.0036
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0021 - val_loss: 0.0041
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0023 - val_loss: 0.0032
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0020 - val_loss: 0.0043
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - loss: 0.0022 - val_l

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - loss: 0.0829 - val_loss: 0.0038
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0043 - val_loss: 0.0026
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0025 - val_loss: 0.0027
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0026 - val_loss: 0.0034
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - loss: 0.0022 - val_loss: 0.0024
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - loss: 0.0023 - val_loss: 0.0026
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0023 - val_loss: 0.0024
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0018 - val_loss: 0.0022
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.0019 - val_loss: 0.0045
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - loss: 0.0023 - val_loss: 0.0024
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 0.0018 - val_loss: 0.0019
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0019 - val_l

...

31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 90ms/step - loss: 0.0554 - val_loss: 0.0045
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - loss: 0.0047 - val_loss: 0.0052
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 0.0032 - val_loss: 0.0055
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 0.0030 - val_loss: 0.0042
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0029 - val_loss: 0.0040
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0025 - val_loss: 0.0029
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - loss: 0.0025 - val_loss: 0.0028
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0023 - val_loss: 0.0073
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0030 - val_loss: 0.0040
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.0023 - val_loss: 0.0025
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 0.0019 - val_loss: 0.0025
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 0.0026 - val_

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - loss: 0.0776 - val_loss: 0.0060
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - loss: 0.0038 - val_loss: 0.0036
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - loss: 0.0026 - val_loss: 0.0030
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0022 - val_loss: 0.0064
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0028 - val_loss: 0.0031
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0021 - val_loss: 0.0027
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - loss: 0.0021 - val_loss: 0.0035
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - loss: 0.0019 - val_loss: 0.0022
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0020 - val_loss: 0.0023
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.0018 - val_loss: 0.0020
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 0.0017 - val_l

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - loss: 0.0246 - val_loss: 0.0051
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step - loss: 0.0020 - val_loss: 0.0060
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0022 - val_loss: 0.0042
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 0.0016 - val_loss: 0.0041
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 0.0018 - val_loss: 0.0074
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - loss: 0.0014 - val_loss: 0.0039
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - loss: 0.0012 - val_loss: 0.0048
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - loss: 0.0013 - val_loss: 0.0036
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 0.0011 - val_loss: 0.0037
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - loss: 0.0011 - val_loss: 0.0040
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 0.0013 - val_loss: 0.0046
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - loss: 0.0011 - val_l

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Flatten, TimeDistributed

from datetime import datetime, timedelta

# Input files
input_files = [
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock A1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock B1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock C1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock D1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock E1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock F1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock G1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock H1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock I1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock J1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock K1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock L1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock M1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock N1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock O1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock P1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock Q1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock R1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock S1.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/offine round/Stock T1.xlsx"
]


for file_path in input_files:
    # Extract file name
    file_name = file_path.split("/")[-1].split(".")[0]

    print(f"Processing file: {file_name}")

    # Load data
    data = pd.read_excel(file_path, sheet_name="Sheet1")

    # Preprocess the Data
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # features to scale
    features_to_scale = ['Close', 'Open', 'High', 'Low', 'Volume']

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[features_to_scale])

    # Training
    def create_sequences(data, window_size=60):
        X, y = [], []
        for i in range(len(data) - window_size):
            X.append(data[i:i+window_size])
            y.append(data[i+window_size, 0])
        return np.array(X), np.array(y)

    window_size = 60
    X, y = create_sequences(scaled_data)

    # Split Data
    train_size = int(len(X) * 0.8)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]

    # LSTM Model
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        LSTM(64, return_sequences=True),
        TimeDistributed(Flatten()),
        LSTM(32),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Train
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        verbose=1
    )

    # Generate Predictions

    # trading dates for 2023
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    trading_dates_2023 = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)
                           if (start_date + timedelta(days=i)).weekday() < 5]

    predictions = []
    last_window = X[-1]

    for _ in range(len(trading_dates_2023)):
        pred = model.predict(np.array([last_window]))[0][0]
        predictions.append(pred)


        last_window = np.roll(last_window, -1, axis=0)
        last_window[-1][0] = pred


    dummy_array = np.zeros((len(predictions), len(features_to_scale)))
    dummy_array[:, 0] = np.array(predictions)

    predicted_prices = scaler.inverse_transform(dummy_array)[:, 0]

    # Save Predictions
    output_file_name = f"{file_name}_predicted_prices_2023.xlsx"

    output_df = pd.DataFrame({
        "Trading Date": trading_dates_2023,
        "Predicted Close": predicted_prices
    })

    output_df.to_excel(output_file_name, index=False)

    print(f"Predictions saved to {output_file_name}")


Processing file: Stock A1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - loss: 0.0576 - val_loss: 0.0063
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0048 - val_loss: 0.0055
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0029 - val_loss: 0.0055
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0024 - val_loss: 0.0054
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0025 - val_loss: 0.0047
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0022 - val_loss: 0.0047
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0020 - val_loss: 0.0039
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0019 - val_loss: 0.0036
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0016 - val_loss: 0.0032
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0017 - val_loss: 0.0030
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - loss: 0.0015 - val_loss: 0.0029
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - loss: 0.0508 - val_loss: 0.0139
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0040 - val_loss: 0.0033
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0026 - val_loss: 0.0038
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0022 - val_loss: 0.0030
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0024 - val_loss: 0.0029
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0023 - val_loss: 0.0027
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0018 - val_loss: 0.0041
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.0016 - val_loss: 0.0030
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0018 - val_loss: 0.0045
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.0016 - val_loss: 0.0025
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0016 - val_loss: 0.0023
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0013 - val_l

...

31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.0766 - val_loss: 0.0086
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0039 - val_loss: 0.0057
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0034 - val_loss: 0.0065
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0031 - val_loss: 0.0035
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0032 - val_loss: 0.0038
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0025 - val_loss: 0.0047
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0023 - val_loss: 0.0037
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0018 - val_loss: 0.0055
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0038 - val_loss: 0.0027
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - loss: 0.0020 - val_loss: 0.0026
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0020 - val_loss: 0.0029
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0019 - val_l

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - loss: 0.0088 - val_loss: 0.0040
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - loss: 0.0030 - val_loss: 0.0032
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0024 - val_loss: 0.0025
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0019 - val_loss: 0.0026
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0015 - val_loss: 0.0040
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 0.0023 - val_loss: 0.0021
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0012 - val_loss: 0.0017
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0012 - val_loss: 0.0021
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - loss: 0.0014 - val_l

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 66ms/step - loss: 0.0284 - val_loss: 0.0190
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0025 - val_loss: 0.0060
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0016 - val_loss: 0.0058
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0013 - val_loss: 0.0051
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: 0.0014 - val_loss: 0.0052
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0015 - val_loss: 0.0053
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.0012 - val_loss: 0.0043
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - loss: 0.0012 - val_loss: 0.0059
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 0.0011 - val_loss: 0.0038
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0012 - val_loss: 0.0042
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - loss: 9.1499e-04 - val_loss: 0.0040
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0013 - v